In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import os

# 读取数据
file_path = 'electricity.csv'  # 根据实际情况修改
if not os.path.exists(file_path):
    raise FileNotFoundError(f"文件 {file_path} 不存在")

data = pd.read_csv(file_path)

# 检查时间戳列是否存在
if 'timestamp' not in data.columns:
    raise ValueError("数据中缺少'timestamp'列")

# 将时间列转换为datetime格式
data['timestamp'] = pd.to_datetime(data['timestamp'])

# ===== 时间戳检查代码 =====
# 1. 检查时间戳是否有缺失值
missing_timestamps = data['timestamp'].isnull().sum()
if missing_timestamps > 0:
    print(f"警告: 时间戳列中有 {missing_timestamps} 个缺失值")

# 2. 检查时间戳是否有重复值
duplicate_timestamps = data['timestamp'].duplicated().sum()
if duplicate_timestamps > 0:
    print(f"警告: 时间戳列中有 {duplicate_timestamps} 个重复值")
    duplicate_indices = data[data['timestamp'].duplicated(keep='first')].index
    print(f"重复的时间戳索引: {duplicate_indices.tolist()}")

# 3. 检查时间间隔是否一致
# 先按时间戳排序
data = data.sort_values('timestamp')

# 计算时间差
time_diff = data['timestamp'].diff()

# 获取最常见的时间间隔(应该是1小时)
expected_interval = time_diff.mode().iloc[0]
print(f"最常见的时间间隔: {expected_interval}")

# 找出不符合预期间隔的时间点(排除第一个点)
irregular_intervals = time_diff[1:][time_diff[1:] != expected_interval]
if not irregular_intervals.empty:
    print(f"发现 {len(irregular_intervals)} 个不规则的时间间隔")
    for idx, interval in irregular_intervals.items():
        current_time = data.loc[idx, 'timestamp']
        previous_time = data.loc[idx-1, 'timestamp'] if idx > 0 else None
        print(f"索引 {idx}: {previous_time} -> {current_time}, 间隔: {interval}")

# 4. 检查是否覆盖了预期的完整时间范围
start_date = data['timestamp'].min()
end_date = data['timestamp'].max()
print(f"数据时间范围: {start_date} 到 {end_date}")

# 创建理想的时间范围(每小时一个点)
ideal_range = pd.date_range(start=start_date, end=end_date, freq='h')  # 修正为'h'

# 检查是否有缺失的时间点
missing_times = set(ideal_range) - set(data['timestamp'])
if missing_times:
    print(f"发现 {len(missing_times)} 个缺失的时间点")
    # 仅打印前10个缺失时间点(如果太多)
    print_times = sorted(list(missing_times))[:10]
    print(f"部分缺失时间点: {print_times}")
    if len(missing_times) > 10:
        print(f"... 以及 {len(missing_times) - 10} 个其他缺失时间点")

# 5. 检查是否有超出预期范围的时间点
expected_start = pd.Timestamp('2016-01-01 00:00:00')
expected_end = pd.Timestamp('2017-12-31 23:00:00')

out_of_range = data[(data['timestamp'] < expected_start) | (data['timestamp'] > expected_end)]
if not out_of_range.empty:
    print(f"发现 {len(out_of_range)} 个超出预期范围的时间点")
    print(out_of_range['timestamp'].tolist())

# ===== 时间戳检查结束 =====

# 设置时间戳为索引
data.set_index('timestamp', inplace=True)

# 选择所有数值列
numeric_cols = data.select_dtypes(include=['number']).columns.tolist()

if not numeric_cols:
    raise ValueError("数据中没有数值列可以处理")

print(f"将处理以下列: {numeric_cols}")

# 保存原始数据的副本，用于后续比较
original_data = data.copy()

# 1. 改进的异常值检测 - 使用更保守的方法
def detect_outliers_improved(df, columns):
    outliers_dict = {}
    
    for col in columns:
        # 计算列的统计量 - 使用更保守的百分位数
        Q1 = df[col].quantile(0.01)  # 更保守的下界
        Q3 = df[col].quantile(0.99)  # 更保守的上界
        IQR = Q3 - Q1
        
        # 设置更宽松的界限
        lower_bound = Q1 - 5 * IQR  # 使用更大乘数
        upper_bound = Q3 + 5 * IQR
        
        # 找出异常值
        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)].index
        outliers_dict[col] = outliers
        
        print(f"检测到{len(outliers)}个异常值在'{col}'列 (修改后的IQR方法)")
    
    return outliers_dict

# 2. 改进的异常值替换方法 - 避免将非零值替换为零
def replace_outliers_improved(df, outliers_dict, original_df):
    df_clean = df.copy()
    
    # 确保所有数值列都是浮点型，避免类型不匹配问题
    for col in numeric_cols:
        df_clean[col] = df_clean[col].astype(float)
    
    for col, idx_list in outliers_dict.items():
        if len(idx_list) > 0:
            for idx in idx_list:
                # 获取当前异常值
                original_value = df.loc[idx, col]
                
                # 获取当前异常值的时间信息
                current_hour = idx.hour
                current_dayofweek = idx.dayofweek
                
                # 找出相似时间模式的非异常值
                similar_times = []
                
                # 查找前后7天同一时间点的数据
                for days_offset in range(-7, 8):
                    if days_offset == 0:
                        continue  # 跳过当前日期
                    
                    similar_time = idx + pd.Timedelta(days=days_offset)
                    if similar_time in df.index and similar_time not in idx_list:
                        similar_times.append(df.loc[similar_time, col])
                
                # 如果找到足够的相似时间点数据
                if len(similar_times) >= 3:
                    # 使用相似时间点的中位数替换异常值
                    replacement_value = np.median(similar_times)
                    
                    # 重要：如果原始值不为零而替换值为零，保留原始值
                    if original_value != 0 and replacement_value == 0:
                        print(f"保留列'{col}'中索引{idx}的非零原始值({original_value})而不是替换为0")
                        df_clean.loc[idx, col] = original_value
                    else:
                        df_clean.loc[idx, col] = replacement_value
                else:
                    # 如果没有足够的相似时间点，尝试使用同一小时段的数据
                    same_hour_data = []
                    for potential_idx in df.index:
                        if (potential_idx.hour == current_hour and 
                            potential_idx not in idx_list and
                            (potential_idx.dayofweek >= 5) == (current_dayofweek >= 5)):  # 工作日/周末匹配
                            same_hour_data.append(df.loc[potential_idx, col])
                    
                    if len(same_hour_data) >= 3:
                        replacement_value = np.median(same_hour_data)
                        
                        # 同样检查：如果原始值不为零而替换值为零，保留原始值
                        if original_value != 0 and replacement_value == 0:
                            print(f"保留列'{col}'中索引{idx}的非零原始值({original_value})而不是替换为0")
                            df_clean.loc[idx, col] = original_value
                        else:
                            df_clean.loc[idx, col] = replacement_value
                    else:
                        # 如果仍然没有足够数据，保留原值
                        df_clean.loc[idx, col] = original_value
    
    return df_clean

# 3. 改进的缺失值填充 - 使用相似时间模式并避免零值替换
def fill_missing_values_improved(df, columns, original_df):
    df_filled = df.copy()
    
    for col in columns:
        # 检查是否有缺失值
        missing_mask = df[col].isnull()
        n_missing = missing_mask.sum()
        
        if n_missing > 0:
            print(f"'{col}'列中有{n_missing}个缺失值")
            
            # 对于每个缺失值
            for idx in df[missing_mask].index:
                # 尝试从原始数据获取值（如果不是NaN）
                if idx in original_df.index and not pd.isna(original_df.loc[idx, col]):
                    original_value = original_df.loc[idx, col]
                    df_filled.loc[idx, col] = original_value
                    print(f"使用原始数据中的值填充'{col}'列索引{idx}的缺失值: {original_value}")
                    continue
                    
                # 获取时间特征
                current_hour = idx.hour
                current_dayofweek = idx.dayofweek
                
                # 查找相似时间模式的有效值
                similar_times = []
                
                # 尝试前后7天同一时间点的数据
                for days_offset in range(-7, 8):
                    if days_offset == 0:
                        continue
                    
                    similar_time = idx + pd.Timedelta(days=days_offset)
                    if similar_time in df.index and not pd.isna(df.loc[similar_time, col]):
                        similar_times.append(df.loc[similar_time, col])
                
                if len(similar_times) >= 3:
                    # 使用相似时间点的中位数填充
                    replacement_value = np.median(similar_times)
                    
                    # 确保不会将非零值替换为零
                    # 对于缺失值，我们没有原始值可比较，所以只要确保填充值不为零即可
                    if replacement_value == 0 and any(v != 0 for v in similar_times):
                        # 如果中位数为零但有非零值，使用非零值的平均值
                        non_zero_values = [v for v in similar_times if v != 0]
                        if non_zero_values:
                            replacement_value = np.mean(non_zero_values)
                    
                    df_filled.loc[idx, col] = replacement_value
                else:
                    # 尝试使用同一小时段的数据
                    same_hour_data = []
                    for potential_idx in df.index:
                        if (potential_idx.hour == current_hour and 
                            not pd.isna(df.loc[potential_idx, col]) and
                            (potential_idx.dayofweek >= 5) == (current_dayofweek >= 5)):
                            same_hour_data.append(df.loc[potential_idx, col])
                    
                    if len(same_hour_data) >= 3:
                        replacement_value = np.median(same_hour_data)
                        
                        # 同样确保不会填充零值
                        if replacement_value == 0 and any(v != 0 for v in same_hour_data):
                            non_zero_values = [v for v in same_hour_data if v != 0]
                            if non_zero_values:
                                replacement_value = np.mean(non_zero_values)
                        
                        df_filled.loc[idx, col] = replacement_value
                    else:
                        # 如果没有足够数据，使用列的非零中位数
                        non_zero_median = df[df[col] != 0][col].median()
                        if pd.isna(non_zero_median):  # 如果没有非零值
                            df_filled.loc[idx, col] = df[col].median()
                        else:
                            df_filled.loc[idx, col] = non_zero_median
    
    return df_filled

# 4. 最终检查函数 - 确保没有非零值被错误地替换为零
def ensure_nonzero_preserved(df_processed, df_original, columns):
    """确保所有列中的非零值不会被错误地设置为零"""
    df_fixed = df_processed.copy()
    
    for col in columns:
        # 检查是否有被设为0的非零原始值
        mask = (df_processed[col] == 0) & (df_original[col] > 0)
        count = mask.sum()
        
        if count > 0:
            print(f"修复：发现{count}个'{col}'列中的非零值被错误地设置为0，正在恢复原值")
            # 恢复原始值
            df_fixed.loc[mask, col] = df_original.loc[mask, col]
    
    return df_fixed

# 执行数据处理流程
# 1. 检测异常值
outliers = detect_outliers_improved(data, numeric_cols)

# 2. 替换异常值
data_clean = replace_outliers_improved(data, outliers, original_data)

# 3. 填充缺失值
data_filled = fill_missing_values_improved(data_clean, numeric_cols, original_data)
# 4. 最终检查 - 确保所有非零值都被保留
data_final = ensure_nonzero_preserved(data_filled, original_data, numeric_cols)

# 检查是否仍有缺失值
missing_counts = data_final[numeric_cols].isnull().sum()
if missing_counts.sum() > 0:
    print("警告: 数据中仍存在缺失值，使用非零中位数填充")
    for col in numeric_cols:
        missing_in_col = missing_counts[col]
        if missing_in_col > 0:
            # 使用非零中位数填充
            non_zero_median = data_final[data_final[col] > 0][col].median()
            if pd.isna(non_zero_median):  # 如果没有非零值
                data_final[col] = data_final[col].fillna(data_final[col].median())
            else:
                data_final[col] = data_final[col].fillna(non_zero_median)
            print(f"填充了'{col}'列中的{missing_in_col}个缺失值")

# 5. 数据类型修复 - 确保没有类型不匹配警告
for col in numeric_cols:
    # 将所有数值列转换为浮点型
    data_final[col] = data_final[col].astype(float)

# 保存处理后的数据
data_final.to_csv('electricity_cleaned.csv')

print("数据处理完成!")

# 验证处理结果
def validate_processing(original, processed, columns):
    """验证处理结果，确保没有非零值被错误地设置为零"""
    all_valid = True
    
    for col in columns:
        # 检查是否有非零值被设为零
        zero_mask = (processed[col] == 0) & (original[col] > 0)
        zero_count = zero_mask.sum()
        
        if zero_count > 0:
            print(f"警告: 列'{col}'中有{zero_count}个非零值被错误地设置为0")
            all_valid = False
            
            # 显示部分被错误修改的值
            if zero_count > 5:
                sample_indices = original[zero_mask].index[:5]
                print("部分示例:")
                for idx in sample_indices:
                    print(f"  索引 {idx}: 原值 {original.loc[idx, col]} -> 处理后 {processed.loc[idx, col]}")
                print(f"  ...以及{zero_count-5}个其他值")
            else:
                for idx in original[zero_mask].index:
                    print(f"  索引 {idx}: 原值 {original.loc[idx, col]} -> 处理后 {processed.loc[idx, col]}")
    
    if all_valid:
        print("验证通过: 所有列中的非零值都被正确保留")
    
    return all_valid

# 验证处理结果
validation_result = validate_processing(original_data, data_final, numeric_cols)

# 提供处理前后的统计比较
print("\n数据处理前后统计比较:")
comparison_stats = pd.DataFrame({
    '原始数据缺失值': original_data[numeric_cols].isnull().sum(),
    '处理后缺失值': data_final[numeric_cols].isnull().sum(),
    '原始数据零值数量': (original_data[numeric_cols] == 0).sum(),
    '处理后零值数量': (data_final[numeric_cols] == 0).sum(),
    '原始数据均值': original_data[numeric_cols].mean(),
    '处理后均值': data_final[numeric_cols].mean(),
    '原始数据中位数': original_data[numeric_cols].median(),
    '处理后中位数': data_final[numeric_cols].median()
})
print(comparison_stats)
# 6. 改进的归一化方法 - 按时间特征分组归一化
def grouped_minmax_scaling(df, columns):
    """按时间特征分组进行Min-Max归一化，保留非零值的特性"""
    result = df.copy()
    scalers = {}  # 存储每个组的缩放器
    
    # 按工作日/周末和小时分组归一化
    for is_weekend in [0, 1]:  # 0=工作日, 1=周末
        for hour in range(24):  # 0-23小时
            # 创建分组掩码
            mask = (df.index.dayofweek >= 5) == (is_weekend == 1)  # 工作日/周末
            mask &= (df.index.hour == hour)  # 小时
            
            # 确保有足够的数据点进行归一化
            if mask.sum() >= 5:  # 至少需要5个数据点
                for col in columns:
                    # 获取当前组的数据
                    group_data = df.loc[mask, col].values.reshape(-1, 1)
                    
                    # 检查组内是否有足够的非零值
                    non_zero_count = (group_data != 0).sum()
                    if non_zero_count < 3:  # 如果非零值太少，跳过归一化
                        print(f"跳过'{col}'列在{hour}时{'周末' if is_weekend else '工作日'}的归一化 - 非零值不足")
                        continue
                    
                    # 创建并拟合缩放器
                    scaler = MinMaxScaler(feature_range=(0, 1))
                    scaler.fit(group_data)
                    
                    # 存储缩放器
                    key = f"{col}_{is_weekend}_{hour}"
                    scalers[key] = scaler
                    
                    # 应用缩放
                    scaled_values = scaler.transform(group_data).flatten()
                    
                    # 重要：确保零值仍然是零
                    zero_mask = (df.loc[mask, col].values == 0)
                    scaled_values[zero_mask] = 0
                    
                    # 更新结果
                    result.loc[mask, col] = scaled_values
            else:
                print(f"跳过{hour}时{'周末' if is_weekend else '工作日'}的归一化 - 数据点不足")
    
    # 处理未归一化的值
    for col in columns:
        # 检查是否有未归一化的值
        not_scaled = pd.isna(result[col]) | (df[col] != result[col])
        if not_scaled.any():
            print(f"'{col}'列中有{not_scaled.sum()}个值未通过分组归一化，使用全局归一化")
            
            # 获取已经归一化和未归一化的索引
            scaled_idx = ~not_scaled
            unscaled_idx = not_scaled
            
            if unscaled_idx.sum() > 0:
                # 创建全局缩放器
                all_data = df[col].values.reshape(-1, 1)
                global_scaler = MinMaxScaler(feature_range=(0, 1))
                global_scaler.fit(all_data)
                
                # 应用全局缩放到未归一化的值
                unscaled_data = df.loc[unscaled_idx, col].values.reshape(-1, 1)
                scaled_values = global_scaler.transform(unscaled_data).flatten()
                
                # 确保零值仍然是零
                zero_mask = (df.loc[unscaled_idx, col].values == 0)
                scaled_values[zero_mask] = 0
                
                # 更新结果
                result.loc[unscaled_idx, col] = scaled_values
    
    return result

# 应用归一化
print("正在进行数据归一化...")
data_normalized = grouped_minmax_scaling(data_final, numeric_cols)

# 最终再次检查，确保零值保持为零
for col in numeric_cols:
    # 确保原始数据中的零值在归一化后仍然是零
    zero_mask = (data_final[col] == 0)
    if zero_mask.any():
        if not (data_normalized.loc[zero_mask, col] == 0).all():
            print(f"修复: 在'{col}'列中的零值在归一化后变为非零")
            data_normalized.loc[zero_mask, col] = 0

# 保存归一化后的数据
data_normalized.to_csv('electricity_normalized.csv')
print("归一化完成!")


最常见的时间间隔: 0 days 01:00:00
数据时间范围: 2016-01-01 00:00:00 到 2017-12-31 23:00:00
将处理以下列: ['Hog_lodging_Brian', 'Hog_lodging_Nikki', 'Hog_lodging_Ora', 'Robin_lodging_Celia', 'Robin_lodging_Elmer', 'Robin_lodging_Renea', 'Hog_health_Hisako', 'Hog_health_Jenny', 'Hog_health_Kesha', 'Rat_health_Gaye', 'Rat_health_Guy', 'Rat_health_Shane', 'Rat_public_Chrissy', 'Eagle_public_Pearle', 'Hog_public_Crystal', 'Hog_public_Kevin', 'Hog_public_Octavia', 'Rat_public_Roma', 'Eagle_office_Ryan', 'Rat_office_Tracy', 'Robin_office_Lindsay', 'Wolf_office_Bobbie', 'Hog_office_Sung', 'Wolf_office_Cary', 'Hog_education_Hallie', 'Hog_education_Haywood', 'Hog_education_Janell', 'Hog_education_Rachael', 'Hog_education_Wayne', 'Robin_education_Zenia']
检测到0个异常值在'Hog_lodging_Brian'列 (修改后的IQR方法)
检测到0个异常值在'Hog_lodging_Nikki'列 (修改后的IQR方法)
检测到0个异常值在'Hog_lodging_Ora'列 (修改后的IQR方法)
检测到0个异常值在'Robin_lodging_Celia'列 (修改后的IQR方法)
检测到0个异常值在'Robin_lodging_Elmer'列 (修改后的IQR方法)
检测到0个异常值在'Robin_lodging_Renea'列 (修改后的IQR方法)
检测到0个异常值在'H

In [6]:
import pandas as pd
import os

# 创建输出目录
os.makedirs('electricity_train_data', exist_ok=True)

# 定义时间范围
splits = {
    'extreme_train': ('2016-01-01 00:00:00', '2016-01-07 23:00:00'),
    'extreme_test': ('2016-01-08 00:00:00', '2016-12-31 23:00:00'),
    'heavy_train': ('2016-01-01 00:00:00', '2016-01-28 23:00:00'),
    'heavy_test': ('2016-01-29 00:00:00', '2016-12-31 23:00:00'),
    'mild_train': ('2016-01-01 00:00:00', '2016-03-31 23:00:00'),
    'mild_test': ('2016-04-01 00:00:00', '2016-12-31 23:00:00')
}

# 检查输入文件是否存在
file_path = 'electricity_source_data/electricity_processed.csv'
if not os.path.exists(file_path):
    raise FileNotFoundError(f"输入文件不存在: {file_path}")

# 读取数据
df = pd.read_csv(file_path)

# 检查时间戳列是否存在
if 'timestamp' not in df.columns:
    raise ValueError("数据中缺少'timestamp'列")

# 确保时间戳列是datetime格式
if not pd.api.types.is_datetime64_any_dtype(df['timestamp']):
    print("将时间戳列转换为datetime格式...")
    df['timestamp'] = pd.to_datetime(df['timestamp'])

# 拆分并保存数据
for split_name, (start, end) in splits.items():
    start_time = pd.Timestamp(start)
    end_time = pd.Timestamp(end)

    mask = (df['timestamp'] >= start_time) & (df['timestamp'] <= end_time)
    split_df = df[mask].copy()

    expected_len = int((end_time - start_time).total_seconds() / 3600) + 1
    if len(split_df) != expected_len:
        print(f"⚠️ {split_name}：期望 {expected_len} 行，实际 {len(split_df)} 行")
        # 输出缺失的时间戳
        all_timestamps = pd.date_range(start=start_time, end=end_time, freq='h')
        missing_timestamps = set(all_timestamps) - set(df['timestamp'])
        if missing_timestamps:
            print(f"   缺失的时间戳数量: {len(missing_timestamps)}")
            if len(missing_timestamps) <= 10:
                print(f"   缺失的时间戳: {sorted(missing_timestamps)}")
            else:
                print(f"   部分缺失的时间戳: {sorted(list(missing_timestamps))[:10]}...")

    split_df.to_csv(f'electricity_train_data/electricity_{split_name}.csv', index=False)
    print(f"✅ 保存文件：electricity_train_data/electricity_{split_name}.csv")

print("\n✅ 所有电力数据拆分完成！")

将时间戳列转换为datetime格式...
✅ 保存文件：electricity_train_data/electricity_extreme_train.csv
✅ 保存文件：electricity_train_data/electricity_extreme_test.csv
✅ 保存文件：electricity_train_data/electricity_heavy_train.csv
✅ 保存文件：electricity_train_data/electricity_heavy_test.csv
✅ 保存文件：electricity_train_data/electricity_mild_train.csv
✅ 保存文件：electricity_train_data/electricity_mild_test.csv

✅ 所有电力数据拆分完成！


## 天气

In [7]:
import pandas as pd

# 读取CSV文件
df = pd.read_csv('weather.csv')

# 定义要保留的地点列表
locations_to_keep = ['Hog', 'Robin', 'Rat', 'Eagle', 'Wolf']

# 定义要删除的列名
columns_to_drop = ['cloudCoverage', 'precipDepth1HR', 'precipDepth6HR']

# 检查要删除的列是否存在于数据中
columns_to_drop = [col for col in columns_to_drop if col in df.columns]

# 筛选数据并删除指定的列
filtered_df = df[df['site_id'].isin(locations_to_keep)]
filtered_df = filtered_df.drop(columns=columns_to_drop)

print(f"已删除以下列: {columns_to_drop}")

# 按地点分割数据并保存为不同的CSV文件
for location in locations_to_keep:
    # 提取当前地点的数据
    location_df = filtered_df[filtered_df['site_id'] == location]
    
    # 跳过没有数据的地点
    if len(location_df) == 0:
        print(f"警告: 没有找到'{location}'的数据")
        continue
    
    # 保存为CSV文件
    output_file = f'{location}.csv'
    location_df.to_csv(output_file, index=False)
    print(f"已保存{location}的天气数据，共{len(location_df)}行，文件名: {output_file}")

# 同时保存一个包含所有筛选地点的合并文件
filtered_df.to_csv('filtered_weather_all.csv', index=False)
print(f"已保存所有筛选地点的合并数据，共{len(filtered_df)}行，文件名: filtered_weather_all.csv")

已删除以下列: ['cloudCoverage', 'precipDepth1HR', 'precipDepth6HR']
已保存Hog的天气数据，共17542行，文件名: Hog.csv
已保存Robin的天气数据，共17516行，文件名: Robin.csv
已保存Rat的天气数据，共17539行，文件名: Rat.csv
已保存Eagle的天气数据，共17536行，文件名: Eagle.csv
已保存Wolf的天气数据，共17505行，文件名: Wolf.csv
已保存所有筛选地点的合并数据，共87638行，文件名: filtered_weather_all.csv


### 这是天气的处理，缺值归一化等等。

In [3]:

import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler

# 定义要处理的文件列表
file_names = [ 'Robin', 'Rat', 'Eagle', 'Wolf']

# 定义不同数据集的时间范围
dataset_types = {
    'extreme_train': ('2016-01-01 00:00:00', '2016-01-07 23:00:00'),
    'extreme_test': ('2016-01-08 00:00:00', '2016-12-31 23:00:00'),
    'heavy_train': ('2016-01-01 00:00:00', '2016-01-28 23:00:00'),
    'heavy_test': ('2016-01-29 00:00:00', '2016-12-31 23:00:00'),
    'mild_train': ('2016-01-01 00:00:00', '2016-03-31 23:00:00'),
    'mild_test': ('2016-04-01 00:00:00', '2016-12-31 23:00:00')
}

# 确保输出目录存在
output_dir = 'weather_train'
os.makedirs(output_dir, exist_ok=True)

# 控制详细日志输出
VERBOSE = False

# 为每个文件执行处理
for file_base in file_names:
    file_path = f'{file_base}.csv'  # 构建完整的文件路径
    
    print(f"\n=== 开始处理 {file_path} ===")
    
    # 检查文件是否存在
    if not os.path.exists(file_path):
        print(f"警告: 文件 {file_path} 不存在，跳过处理")
        continue
    
    # 读取CSV文件
    try:
        data = pd.read_csv(file_path)
        print(f"成功读取文件，包含 {len(data)} 行数据")
    except Exception as e:
        print(f"读取文件 {file_path} 时出错: {e}")
        continue
    
    # 将时间列转换为datetime格式
    if 'timestamp' in data.columns:
        data['timestamp'] = pd.to_datetime(data['timestamp'])
    else:
        print(f"错误: {file_path} 中缺少'timestamp'列")
        continue
    
    # ===== 时间戳检查代码 =====
    print("=== 时间戳完整性检查 ===")
    
    # 1. 检查时间戳是否有缺失值
    missing_timestamps = data['timestamp'].isnull().sum()
    if missing_timestamps > 0:
        print(f"警告: 时间戳列中有 {missing_timestamps} 个缺失值")
    
    # 2. 检查时间戳是否有重复值
    duplicate_timestamps = data['timestamp'].duplicated().sum()
    if duplicate_timestamps > 0:
        print(f"警告: 时间戳列中有 {duplicate_timestamps} 个重复值")
        if VERBOSE:
            duplicate_indices = data[data['timestamp'].duplicated(keep='first')].index
            print(f"重复的时间戳索引: {duplicate_indices.tolist()}")
    
    # 3. 检查时间间隔是否一致
    # 先按时间戳排序
    data = data.sort_values('timestamp')
    
    # 计算时间差
    time_diff = data['timestamp'].diff()
    
    # 获取最常见的时间间隔(应该是1小时)
    expected_interval = time_diff.mode().iloc[0]
    print(f"最常见的时间间隔: {expected_interval}")
    
    # 找出不符合预期间隔的时间点(排除第一个点)
    irregular_intervals = time_diff[1:][time_diff[1:] != expected_interval]
    if not irregular_intervals.empty:
        print(f"发现 {len(irregular_intervals)} 个不规则的时间间隔")
        if VERBOSE:
            for idx, interval in irregular_intervals.items():
                current_time = data.loc[idx, 'timestamp']
                previous_time = data.loc[idx-1, 'timestamp'] if idx > 0 else None
                print(f"索引 {idx}: {previous_time} -> {current_time}, 间隔: {interval}")
    
    # 4. 检查是否覆盖了预期的完整时间范围
    start_date = data['timestamp'].min()
    end_date = data['timestamp'].max()
    print(f"数据时间范围: {start_date} 到 {end_date}")
    
    # 创建理想的时间范围(每小时一个点)
    ideal_range = pd.date_range(start=start_date, end=end_date, freq='h')
    
    # 检查是否有缺失的时间点
    missing_times = set(ideal_range) - set(data['timestamp'])
    if missing_times:
        print(f"发现 {len(missing_times)} 个缺失的时间点")
        if VERBOSE:
            # 仅打印前10个缺失时间点(如果太多)
            print_times = sorted(list(missing_times))[:10]
            print(f"部分缺失时间点: {print_times}")
            if len(missing_times) > 10:
                print(f"... 以及 {len(missing_times) - 10} 个其他缺失时间点")
    
    # 5. 检查闰年2月29日是否存在
    feb_29_date = pd.Timestamp('2016-02-29').date()
    feb_29_data = data[data['timestamp'].dt.date == feb_29_date]
    if len(feb_29_data) == 0:
        print("警告: 数据中缺少2016年2月29日(闰年)的数据")
    elif len(feb_29_data) < 24:
        print(f"警告: 2016年2月29日数据不完整，只有{len(feb_29_data)}小时，应该有24小时")
    else:
        print(f"2016年2月29日(闰年)数据完整，共有{len(feb_29_data)}小时的数据")
    
    # 6. 处理任何时间戳问题
    # 处理重复的时间戳
    if duplicate_timestamps > 0:
        print("正在处理重复的时间戳...")
        # 对于重复的时间戳，保留第一个记录
        data = data.drop_duplicates(subset='timestamp', keep='first')

    # 处理缺失的时间点 - 创建带有标记的完整时间序列
    if missing_times:
        print("正在处理缺失的时间点...")
        # 创建一个完整的时间序列DataFrame
        complete_index = pd.DataFrame(index=ideal_range)
        # 将原始数据与完整索引合并
        data = data.set_index('timestamp').join(complete_index, how='right')
        # 添加一个标记列，标识哪些行是因缺失而插入的
        data['is_imputed'] = data.isnull().all(axis=1)
        # 重置索引，使timestamp成为列
        data = data.reset_index().rename(columns={'index': 'timestamp'})
    else:
        # 如果没有缺失时间点，也添加标记列，全部标记为False
        data['is_imputed'] = False
    
    # 删除site_id列（如果存在）
    if 'site_id' in data.columns:
        print("删除site_id列...")
        data = data.drop(columns=['site_id'])
    
    # 保存原始数据的副本，用于后续比较
    original_data = data.copy()
    
    # 选择需要处理的数值列
    numeric_cols = data.select_dtypes(include=['number']).columns.tolist()
    # 从数值列中移除is_imputed标记列
    if 'is_imputed' in numeric_cols:
        numeric_cols.remove('is_imputed')
    
    print(f"\n处理以下列: {numeric_cols}")
    
    # 查看缺失值情况
    missing_stats = data[numeric_cols].isnull().sum()
    print("\n各列缺失值数量:")
    print(missing_stats)
    
    # 1. 改进的异常值检测 - 参考电力数据处理方法，使用更保守的方法
    def detect_outliers_improved(df, columns, verbose=False):
        """使用更保守的方法检测异常值，区分原始0值和缺失值"""
        outliers_dict = {}
        
        for col in columns:
            # 首先只考虑非缺失的数据
            valid_data = df[~df[col].isnull()]
            
            # 分别处理零值和非零值
            non_zero_data = valid_data[valid_data[col] > 0]
            
            if len(non_zero_data) < 10:  # 如果非零数据太少，跳过此列
                if verbose:
                    print(f"列'{col}'中非零值太少，跳过异常值检测")
                outliers_dict[col] = []
                continue
                
            # 计算非零数据的统计量 - 使用更保守的百分位数
            Q1 = non_zero_data[col].quantile(0.01)  # 更保守的下界
            Q3 = non_zero_data[col].quantile(0.99)  # 更保守的上界
            IQR = Q3 - Q1
            
            # 设置更宽松的界限，参考电力数据处理方法
            lower_bound = max(Q1 - 5 * IQR, 0)  # 使用更大乘数，确保下界不小于0
            upper_bound = Q3 + 5 * IQR
            
            # 对非零值进行异常检测
            non_zero_outliers = valid_data[(valid_data[col] < lower_bound) & 
                                          (valid_data[col] > 0) | 
                                          (valid_data[col] > upper_bound)].index.tolist()
            
            # 对零值进行特殊处理
            zero_outliers = []
            zero_values = valid_data[valid_data[col] == 0]
            
            if len(zero_values) > 0:
                # 检查每个零值是否可能是异常的
                for idx in zero_values.index:
                    # 获取时间特征
                    current_timestamp = df.loc[idx, 'timestamp']
                    current_hour = current_timestamp.hour
                    current_month = current_timestamp.month
                    current_dayofweek = current_timestamp.dayofweek
                    
                    # 查找相似时间点的数据
                    similar_times_data = df[(df['timestamp'].dt.hour == current_hour) & 
                                        (df['timestamp'].dt.month == current_month) & 
                                        (~df[col].isnull()) &
                                        (df[col] > 0)]
                    
                    # 如果相似时间点通常有非零值，则这个零可能是异常的
                    if len(similar_times_data) >= 5 and similar_times_data[col].mean() > 0.5:
                        zero_outliers.append(idx)
            
            # 合并非零异常值和异常零值
            outliers_dict[col] = non_zero_outliers + zero_outliers
            
            print(f"检测到{len(outliers_dict[col])}个异常值在'{col}'列 (包括{len(zero_outliers)}个可疑的零值)")
            if len(non_zero_outliers) > 0 and verbose:
                print(f"  非零值范围: 下界={lower_bound:.4f}, 上界={upper_bound:.4f}")
        
        return outliers_dict
    
    # 2. 改进的异常值替换方法 - 参考电力数据处理，避免将非零值替换为零
    def replace_outliers_improved(df, outliers_dict, verbose=False):
        """替换异常值，特别注意避免将非零值错误地替换为零"""
        df_clean = df.copy()
        
        # 确保所有数值列都是浮点型，避免类型不匹配问题
        for col in numeric_cols:
            df_clean[col] = df_clean[col].astype(float)
        
        total_replaced = 0
        for col, idx_list in outliers_dict.items():
            col_replaced = 0
            if len(idx_list) > 0:
                for idx in idx_list:
                    # 获取当前异常值
                    original_value = df.loc[idx, col]
                    
                    # 获取时间特征
                    current_timestamp = df.loc[idx, 'timestamp']
                    current_hour = current_timestamp.hour
                    current_month = current_timestamp.month
                    current_dayofweek = current_timestamp.dayofweek
                    
                    # 查找前后7天同一时间点的数据（参考电力数据处理方法）
                    similar_values = []
                    for days_offset in range(-7, 8):
                        if days_offset == 0:
                            continue  # 跳过当前日期
                        
                        offset_date = current_timestamp + pd.Timedelta(days=days_offset)
                        similar_idx = df[(df['timestamp'].dt.hour == offset_date.hour) & 
                                        (df['timestamp'].dt.day == offset_date.day) & 
                                        (df['timestamp'].dt.month == offset_date.month)].index

                        if len(similar_idx) > 0 and similar_idx[0] not in outliers_dict[col]:
                            similar_value = df.loc[similar_idx[0], col]
                            if not pd.isna(similar_value):
                                similar_values.append(similar_value)
                    
                    # 如果找到足够的相似时间点数据
                    if len(similar_values) >= 3:
                        # 使用相似时间点的中位数替换异常值
                        replacement_value = np.median(similar_values)
                        
                        # 重要：如果原始值不为零而替换值为零，保留原始值
                        if original_value > 0 and replacement_value == 0:
                            if verbose:
                                print(f"保留列'{col}'中索引{idx}的非零原始值({original_value})而不是替换为0")
                        else:
                            df_clean.loc[idx, col] = replacement_value
                            col_replaced += 1
                            if verbose:
                                print(f"替换列'{col}'索引{idx}的异常值: {original_value} -> {replacement_value}")
                    else:
                        # 如果没有足够的相似时间点，尝试使用同一小时段的数据
                        same_hour_data = df[(df['timestamp'].dt.hour == current_hour) & 
                                          (~df[col].isnull()) & 
                                          (~df.index.isin(outliers_dict[col]))]
                        
                        if len(same_hour_data) >= 3:
                            replacement_value = same_hour_data[col].median()
                            
                            # 同样检查：如果原始值不为零而替换值为零，保留原始值
                            if original_value > 0 and replacement_value == 0:
                                if verbose:
                                    print(f"保留列'{col}'中索引{idx}的非零原始值({original_value})而不是替换为0")
                            else:
                                df_clean.loc[idx, col] = replacement_value
                                col_replaced += 1
                                if verbose:
                                    print(f"使用同时段数据替换列'{col}'索引{idx}的异常值: {original_value} -> {replacement_value}")
                        else:
                            # 如果仍然没有足够数据，使用非零值的中位数
                            non_zero_values = df[df[col] > 0][col].dropna()
                            if len(non_zero_values) > 0:
                                replacement_value = non_zero_values.median()
                                
                                # 再次检查是否会将非零值替换为零
                                if original_value > 0 and replacement_value == 0:
                                    if verbose:
                                        print(f"保留列'{col}'中索引{idx}的非零原始值({original_value})而不是替换为0")
                                else:
                                    df_clean.loc[idx, col] = replacement_value
                                    col_replaced += 1
                                    if verbose:
                                        print(f"使用非零中位数替换列'{col}'索引{idx}的异常值: {original_value} -> {replacement_value}")
                            else:
                                # 极端情况：保留原值
                                if verbose:
                                    print(f"警告: 无法找到合适的替换值，保留列'{col}'索引{idx}的原始值: {original_value}")
                
                print(f"列'{col}'中替换了{col_replaced}个异常值")
                total_replaced += col_replaced
        
        print(f"总共替换了{total_replaced}个异常值")
        return df_clean
    
    # 3. 改进的缺失值填充 - 参考电力数据处理，避免零值填充
    def fill_missing_values_improved(df, columns, verbose=False):
        """智能填充缺失值，避免使用零值填充，保持数据特征和时间相关性"""
        df_filled = df.copy()
        total_filled = 0
        
        for col in columns:
            # 检查是否有缺失值
            missing_mask = df[col].isnull()
            n_missing = missing_mask.sum()
            
            if n_missing > 0:
                print(f"'{col}'列中有{n_missing}个缺失值")
                col_filled = 0
                
                # 获取所有缺失行的信息
                missing_indices = df[missing_mask].index
                
                for i, idx in enumerate(missing_indices):
                    # 获取时间特征
                    current_timestamp = df.loc[idx, 'timestamp']
                    current_hour = current_timestamp.hour
                    current_month = current_timestamp.month
                    current_dayofweek = current_timestamp.dayofweek
                    is_imputed = df.loc[idx, 'is_imputed']
                    
                    # 初始化similar_values列表
                    similar_values = []
                    
                    # 1. 尝试使用前后7天同一时间点的数据（参考电力数据处理）
                    for days_offset in range(-7, 8):
                        if days_offset == 0:
                            continue  # 跳过当前日期
                        
                        offset_date = current_timestamp + pd.Timedelta(days=days_offset)
                        similar_idx = df[(df['timestamp'].dt.hour == offset_date.hour) & 
                                       (df['timestamp'].dt.day == offset_date.day) & 
                                       (df['timestamp'].dt.month == offset_date.month) &
                                       (~df[col].isnull())].index
                        
                        if len(similar_idx) > 0:
                            similar_value = df.loc[similar_idx[0], col]
                            if not pd.isna(similar_value) and similar_value > 0:  # 只考虑非零值
                                similar_values.append(similar_value)
                    
                    # 如果找到足够的相似时间点数据
                    if len(similar_values) >= 3:
                        # 使用相似时间点的中位数填充
                        fill_value = np.median(similar_values)
                        df_filled.loc[idx, col] = fill_value
                        col_filled += 1
                        if verbose and (i % 100 == 0 or i == n_missing - 1):
                            print(f"使用相似时间点的中位数填充列'{col}'索引{idx}的缺失值: {fill_value}")
                        continue
                    
                    # 2. 如果找不到足够的相似时间点，尝试使用线性插值（对于插入的缺失时间点）
                    if is_imputed:
                        try:
                            # 找出前后最近的非缺失值时间点
                            prev_data = df[(df['timestamp'] < current_timestamp) & 
                                         (~df[col].isnull())].sort_values('timestamp', ascending=False)
                            next_data = df[(df['timestamp'] > current_timestamp) & 
                                         (~df[col].isnull())].sort_values('timestamp')
                            
                            if not prev_data.empty and not next_data.empty:
                                prev_idx = prev_data.index[0]
                                next_idx = next_data.index[0]

                                prev_value = df.loc[prev_idx, col]
                                next_value = df.loc[next_idx, col]
                                
                                # 只有当两个值都不为零或者至少有一个非零值时才进行线性插值
                                if prev_value > 0 or next_value > 0:
                                    prev_time = df.loc[prev_idx, 'timestamp']
                                    next_time = df.loc[next_idx, 'timestamp']
                                    
                                    # 计算时间差
                                    time_diff_total = (next_time - prev_time).total_seconds()
                                    time_diff_current = (current_timestamp - prev_time).total_seconds()
                                    
                                    # 线性插值
                                    if time_diff_total > 0:
                                        weight = time_diff_current / time_diff_total
                                        interpolated_value = prev_value + (next_value - prev_value) * weight
                                        
                                        # 确保插值结果不为负
                                        interpolated_value = max(0, interpolated_value)
                                        
                                        df_filled.loc[idx, col] = interpolated_value
                                        col_filled += 1
                                        if verbose and (i % 100 == 0 or i == n_missing - 1):
                                            print(f"使用线性插值填充列'{col}'索引{idx}的缺失值: {interpolated_value}")
                                        continue
                        except Exception as e:
                            if verbose:
                                print(f"线性插值过程中出错: {e}")
                    
                    # 3. 如果前两种方法都失败，尝试使用相似时间模式的数据
                    similar_times = df[
                        (df['timestamp'].dt.month == current_month) & 
                        (df['timestamp'].dt.hour == current_hour) & 
                        ((df['timestamp'].dt.dayofweek >= 5) == (current_dayofweek >= 5)) &
                        (~df[col].isnull()) &
                        (df[col] > 0)  # 只考虑非零值
                    ]
                    
                    if len(similar_times) >= 3:
                        # 使用相似值的中位数填充缺失值
                        fill_value = similar_times[col].median()
                        df_filled.loc[idx, col] = fill_value
                        col_filled += 1
                        if verbose and (i % 100 == 0 or i == n_missing - 1):
                            print(f"使用相似时间模式的中位数填充列'{col}'索引{idx}的缺失值: {fill_value}")
                    else:
                        # 4. 如果没有足够相似值，尝试放宽条件：只匹配小时
                        broader_similar_times = df[
                            (df['timestamp'].dt.hour == current_hour) &
                            (~df[col].isnull()) &
                            (df[col] > 0)  # 只考虑非零值
                        ]
                        
                        if len(broader_similar_times) >= 3:
                            fill_value = broader_similar_times[col].median()
                            df_filled.loc[idx, col] = fill_value
                            col_filled += 1
                            if verbose and (i % 100 == 0 or i == n_missing - 1):
                                print(f"使用同时段数据的中位数填充列'{col}'索引{idx}的缺失值: {fill_value}")
                        else:
                            # 5. 如果仍然没有足够数据，使用所有非零值的中位数
                            non_zero_values = df[df[col] > 0][col].dropna()
                            if len(non_zero_values) > 0:
                                fill_value = non_zero_values.median()
                                df_filled.loc[idx, col] = fill_value
                                col_filled += 1
                                if verbose and (i % 100 == 0 or i == n_missing - 1):
                                    print(f"使用所有非零值的中位数填充列'{col}'索引{idx}的缺失值: {fill_value}")
                            else:
                                # 极端情况：使用一个小的非零值（而不是0）
                                fill_value = 0.01  # 使用一个很小的非零值
                                df_filled.loc[idx, col] = fill_value
                                col_filled += 1
                                if verbose:
                                    print(f"警告: 无法找到合适的填充值，使用小的非零值{fill_value}填充列'{col}'索引{idx}的缺失值")
                
                print(f"列'{col}'中填充了{col_filled}个缺失值")
                total_filled += col_filled
        
        print(f"总共填充了{total_filled}个缺失值")
        return df_filled
    
    # 4. 最终检查函数 - 从电力数据处理借鉴，确保没有非零值被错误地替换为零
    def ensure_nonzero_preserved(df_processed, df_original, columns):
        """确保所有列中的非零值不会被错误地设置为零"""
        df_fixed = df_processed.copy()
        total_fixed = 0
        
        for col in columns:
            # 检查是否有被设为0的非零原始值
            mask = (df_processed[col] == 0) & (df_original[col] > 0)
            count = mask.sum()
            
            if count > 0:
                print(f"修复：发现{count}个'{col}'列中的非零值被错误地设置为0，正在恢复原值")
                # 恢复原始值
                df_fixed.loc[mask, col] = df_original.loc[mask, col]
                total_fixed += count
        
        if total_fixed > 0:
            print(f"总共修复了{total_fixed}个被错误设置为0的非零值")
        else:
            print("没有发现被错误设置为0的非零值")
            
        return df_fixed
    
    def normalize_all_numeric_data(df, columns):
        """对数据进行归一化处理，适当处理负值、零值和正值"""
        print("\n=== 执行数据归一化（全面处理） ===")
        df_normalized = df.copy()
        
        for col in columns:
            try:
                # 检查列的数据特性
                valid_values = df[col].dropna()
                min_value = valid_values.min()
                max_value = valid_values.max()
                zero_count = (valid_values == 0).sum()
                has_negative = (valid_values < 0).any()
                total_count = len(valid_values)
                
                print(f"列 '{col}': 范围=[{min_value:.4f}, {max_value:.4f}], 零值数量={zero_count}, 含负值={has_negative}")
                
                if total_count > 0:
                    # 根据数据特性选择不同的归一化策略
                    if col in ['airTemperature', 'dewTemperature'] or has_negative:
                        # 对可能包含负值的温度数据使用标准MinMaxScaler
                        scaler = MinMaxScaler(feature_range=(0, 1))
                        values_2d = valid_values.values.reshape(-1, 1)
                        scaler.fit(values_2d)
                        
                        # 对所有非缺失值进行归一化
                        non_missing_mask = ~df[col].isnull()
                        if any(non_missing_mask):
                            values_to_transform = df.loc[non_missing_mask, col].values.reshape(-1, 1)
                            normalized_values = scaler.transform(values_to_transform).flatten()
                            df_normalized.loc[non_missing_mask, col] = normalized_values
                        
                        print(f"归一化列 '{col}' (包含负值): 范围从 [{min_value:.4f}, {max_value:.4f}] 归一化到 [0, 1]")
                    else:
                        # 对主要为正值的数据使用保留零值的方法
                        non_zero_values = valid_values[valid_values != 0]
                        if len(non_zero_values) > 0:
                            scaler = MinMaxScaler(feature_range=(0.01, 1))
                            non_zero_values_2d = non_zero_values.values.reshape(-1, 1)
                            scaler.fit(non_zero_values_2d)
                            
                            # 对非零非缺失值进行归一化
                            non_zero_mask = (df[col] != 0) & (~df[col].isnull())
                            if any(non_zero_mask):
                                values_to_transform = df.loc[non_zero_mask, col].values.reshape(-1, 1)
                                normalized_values = scaler.transform(values_to_transform).flatten()
                                
                                # 更新归一化后的值
                                normalized_col = df[col].copy()
                                normalized_col.loc[non_zero_mask] = normalized_values
                                df_normalized[col] = normalized_col
                            
                            print(f"归一化列 '{col}' (保留零值): 非零值范围从 [{non_zero_values.min():.4f}, {non_zero_values.max():.4f}] 归一化到 [0.01, 1]")
                        else:
                            print(f"警告: 列 '{col}' 没有非零值，跳过归一化")
                else:
                    print(f"警告: 列 '{col}' 没有有效值，跳过归一化")
            except Exception as e:
                print(f"归一化列 '{col}' 时出错: {str(e)}")
                # 保持原始值
        
        return df_normalized
    
    # 执行数据处理流程
    print("\n=== 开始数据处理 ===")
    
    # 1. 检测异常值
    print("\n检测异常值...")
    outliers = detect_outliers_improved(data, numeric_cols, verbose=VERBOSE)
    
    # 2. 替换异常值
    print("\n替换异常值...")
    data_cleaned = replace_outliers_improved(data, outliers, verbose=VERBOSE)
    
    # 3. 填充缺失值
    print("\n填充缺失值...")
    data_filled = fill_missing_values_improved(data_cleaned, numeric_cols, verbose=VERBOSE)
    
    # 4. 最终检查 - 确保非零值不被错误地替换为零
    print("\n执行最终检查...")
    data_checked = ensure_nonzero_preserved(data_filled, original_data, numeric_cols)
    
    # 5. 数据归一化 - 使用全面处理的方法
    print("\n执行数据归一化...")
    data_normalized = normalize_all_numeric_data(data_checked, numeric_cols)
    
    # 删除标记列（处理完成后不再需要）
    if 'is_imputed' in data_normalized.columns:
        data_normalized = data_normalized.drop(columns=['is_imputed'])
    
    # 检查处理后的数据质量
    print("\n=== 数据处理完成 ===")
    print(f"原始数据中缺失值总数: {original_data[numeric_cols].isnull().sum().sum()}")
    print(f"处理后数据中缺失值总数: {data_normalized[numeric_cols].isnull().sum().sum()}")
    
    # 计算原始数据和处理后数据的0值数量
    original_zeros = sum((original_data[numeric_cols] == 0).sum())
    processed_zeros = sum((data_normalized[numeric_cols] == 0).sum())
    print(f"原始数据中0值总数: {original_zeros}")
    print(f"处理后数据中0值总数: {processed_zeros}")
    
    # 验证处理结果
    def validate_processing(original, processed, columns):
        """验证处理结果，确保没有非零值被错误地设置为零"""
        all_valid = True
        
        for col in columns:
            # 检查是否有非零值被设为零
            mask = (processed[col] == 0) & (original[col] > 0) & (~original[col].isnull())
            zero_count = mask.sum()
            
            if zero_count > 0:
                print(f"警告: 列'{col}'中有{zero_count}个非零值被错误地设置为0")
                all_valid = False
                
                if VERBOSE:
                    # 显示部分被错误修改的值
                    problem_indices = original[mask].index[:5]  # 最多显示5个
                    for idx in problem_indices:
                        print(f"  索引 {idx}: 原值 {original.loc[idx, col]} -> 新值 0")
            
            # 检查是否有缺失值
            missing_count = processed[col].isnull().sum()
            if missing_count > 0:
                print(f"警告: 列'{col}'中仍有{missing_count}个缺失值")
                all_valid = False
        
        if all_valid:
            print("验证通过: 所有非零值都被正确保留，没有缺失值")
        
        return all_valid
    
    # 执行验证
    print("\n=== 验证处理结果 ===")
    validation_result = validate_processing(original_data, data_normalized, numeric_cols)
    
    # 5. 为不同的数据集划分并保存数据
    print("\n=== 划分并保存数据集 ===")
    for dataset_name, (start_time, end_time) in dataset_types.items():
        # 根据时间范围筛选数据
        mask = (data_normalized['timestamp'] >= start_time) & (data_normalized['timestamp'] <= end_time)
        subset = data_normalized[mask].copy()
        
        if len(subset) == 0:
            print(f"警告: {dataset_name}数据集为空，跳过保存")
            continue
        
        print(f"划分{dataset_name}数据集，包含{len(subset)}行数据")
        
        # 保存处理后的数据
        output_file = f"{output_dir}/{file_base}_{dataset_name}.csv"
        subset.to_csv(output_file, index=False)
        print(f"已保存到{output_file}")


=== 开始处理 Robin.csv ===
成功读取文件，包含 17516 行数据
=== 时间戳完整性检查 ===
警告: 时间戳列中有 2 个重复值
最常见的时间间隔: 0 days 01:00:00
发现 19 个不规则的时间间隔
数据时间范围: 2016-01-01 00:00:00 到 2017-12-31 23:00:00
发现 30 个缺失的时间点
2016年2月29日(闰年)数据完整，共有24小时的数据
正在处理重复的时间戳...
正在处理缺失的时间点...
删除site_id列...

处理以下列: ['airTemperature', 'dewTemperature', 'seaLvlPressure', 'windDirection', 'windSpeed']

各列缺失值数量:
airTemperature    31
dewTemperature    31
seaLvlPressure    91
windDirection     36
windSpeed         30
dtype: int64

=== 开始数据处理 ===

检测异常值...
检测到11个异常值在'airTemperature'列 (包括11个可疑的零值)
检测到58个异常值在'dewTemperature'列 (包括58个可疑的零值)
检测到0个异常值在'seaLvlPressure'列 (包括0个可疑的零值)
检测到40个异常值在'windDirection'列 (包括40个可疑的零值)
检测到40个异常值在'windSpeed'列 (包括40个可疑的零值)

替换异常值...
列'airTemperature'中替换了11个异常值
列'dewTemperature'中替换了58个异常值
列'windDirection'中替换了40个异常值
列'windSpeed'中替换了40个异常值
总共替换了149个异常值

填充缺失值...
'airTemperature'列中有31个缺失值
列'airTemperature'中填充了31个缺失值
'dewTemperature'列中有31个缺失值
列'dewTemperature'中填充了31个缺失值
'seaLvlPressure'列中有91个缺失值
列'seaLvlPressure'中填充了91个缺失值